In [ ]:
# this file is for temporarily matching figuring out how to match what we have in the model to the lit data 
# easier to debugg in here!


In [9]:
#import polars as pl
import numpy as np
from matplotlib import pyplot as plt
import os
import json
import pandas as pd
#import seaborn as sns
from scipy.sparse import csr_matrix

In [25]:
# Load in the data sources that we currently use: 
# TODO: not sure why, but I cannot figure out how to import these not using the absolute path...

Nend_data = pd.read_table("/Users/miagrahn/wcEcoli/reconstruction/ecoli/flat/protein_half_lives_n_end_rule.tsv", comment="#")
exp_data = pd.read_table("/Users/miagrahn/wcEcoli/reconstruction/ecoli/flat/protein_half_lives_measured.tsv", comment="#")
ML_data = pd.read_table("/Users/miagrahn/wcEcoli/reconstruction/ecoli/flat/protein_half_lives_pulsed_silac.tsv", comment="#")

In [32]:
# Load in the experimental data from the new paper (Gupta et al., 2023), Table S1 (https://www.biorxiv.org/content/10.1101/2022.08.01.502339v2.full): 

# Measured half lives (from Gupta et al., 2023 bioRxiv)
new_data = pd.read_table("/Users/miagrahn/wcEcoli/reconstruction/ecoli/flat/protein_half_lives_gupta.tsv", comment="#")
#TODO: figure out to read out the new data with out having to use my aboslute file path. NOt sure why it isnt working....

## figure out what all the current monomers in the model are:

In [29]:
# N-end rule calculated protein degredation rates

# Half lives of protein monomers are determined by the identity of their amino-terminal residues. Slow degradation is assumed for P and U. (Tobias et al, 1986)
Nend_monomers = Nend_data.aa_id
# TODO: this is the only one I am sort of not sure how stuff in it is defined? @Nora do you know what these are and how they could relate to protein/monomer names?
Nend_monomers

0      L-ALPHA-ALANINE[c]
1                  ARG[c]
2                  ASN[c]
3          L-ASPARTATE[c]
4                  CYS[c]
5                  GLT[c]
6                  GLN[c]
7                  GLY[c]
8                  HIS[c]
9                  ILE[c]
10                 LEU[c]
11                 LYS[c]
12                 MET[c]
13                 PHE[c]
14                 PRO[c]
15                 SER[c]
16                 THR[c]
17                 TRP[c]
18                 TYR[c]
19    L-SELENOCYSTEINE[c]
20                 VAL[c]
Name: aa_id, dtype: object

In [30]:
# Experimentally measured protein degredation rates 

# Measured half lives (from Macklin et al., 2020)
exp_monomers = exp_data.id
exp_monomers

0                DCUR-MONOMER
1    DETHIOBIOTIN-SYN-MONOMER
2             EG10863-MONOMER
3              CARBPSYN-SMALL
4             EG10743-MONOMER
5          GLUTCYSLIG-MONOMER
6         CDPDIGLYSYN-MONOMER
Name: id, dtype: object

In [31]:
# ML DATA

# Generated by reconstruction/ecoli/scripts/protein_half_lives/convert_to_flat.py on Mon May  9 13:39:16 2022 
# using the methods in https://pubmed.ncbi.nlm.nih.gov/33531410/
ML_monomers = ML_data.id
ML_monomers

0                   2-ISOPROPYLMALATESYN-MONOMER
1       2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER
2       3-CH3-2-OXOBUTANOATE-OH-CH3-XFER-MONOMER
3                 3-ISOPROPYLMALDEHYDROG-MONOMER
4                   3-OXOACYL-ACP-REDUCT-MONOMER
                          ...                   
1088                                        YIDC
1089                                YJCX-MONOMER
1090                                YJJK-MONOMER
1091                                YLIB-MONOMER
1092                                ZNUA-MONOMER
Name: id, Length: 1093, dtype: object

In [ ]:
# New Data
new_monomers = new_data["Protein name"]
new_genes = new_data["Gene name"]

## Map the monomer names in the current data to gene names:

In [66]:
# Obtain the gene id and monomer id mappings: 

rnas = pd.read_table("/Users/miagrahn/wcEcoli/reconstruction/ecoli/flat/rnas.tsv", comment="#")

m_ids = rnas.monomer_ids
monomer_ids = []
for i in range(len(m_ids)):
	m = m_ids[i]
	m = m.replace("[",""); m = m.replace("]","")
	m = m.replace('"','')
	monomer_ids.append(m)
	
gene_ids = rnas.gene_id
gene_CNs = rnas.common_name

# make organized libraries of monomer names to indexes: 
monomer_idx_dict = {id: i for i, id in enumerate(monomer_ids)}
idx_monomer_dict = {idx: i for i, idx in monomer_idx_dict.items()}

gene_idx_dict = {id: i for i, id in enumerate(gene_ids)}
idx_gene_dict = {idx: i for i, idx in gene_idx_dict.items()}

GCN_idx_dict = {id: i for i, id in enumerate(gene_CNs)}
idx_GCN_dict = {idx: i for i, idx in GCN_idx_dict.items()}


In [ ]:
# Figure out which gene each monomer in the N end rule data set corresponds to: 

# TODO: figure out the amino acid part of this

In [74]:
# Figure out which gene each monomer in the experimental data set corresponds to: 
exp_match_Pids = []
exp_match_idxs = [] 
exp_match_Gids = []
exp_match_GCNids = []

too_many_exp_match_Pidxs = []
non_exp_match_Pids = []
non_exp_match_Pidxs = []

for i in range(len(exp_monomers)):
	m = str(exp_monomers[i])
	index = [name for key, name in monomer_idx_dict.items() if m in key]
	if len(index) > 1: 
		print("too many?")
		too_many_exp_match_Pidxs.append(index)
	if len(index) == 0:
		index = index[0]
		non_exp_match_Pids.append(m)
		non_exp_match_Pidxs.append(i)
	else: 
		index = index[0]
		exp_match_Pids.append(m)
		exp_match_idxs.append(i)
		gene_id = idx_gene_dict.get(index)
		exp_match_Gids.append(gene_id)
		gene_CN = idx_GCN_dict.get(index)
		exp_match_GCNids.append(gene_CN)
		
print(len(exp_match_Pids))

7


In [90]:
# Figure out which gene each monomer in the ML data set corresponds to: 
ML_match_Pids = []
ML_match_idxs = [] 
ML_match_Gids = []
ML_match_GCNids = []

too_many_ML_match_Pids = []
too_many_ML_match_Pidxs = []
non_ML_match_Pids = []
non_ML_match_Pidxs = []

for i in range(len(ML_monomers)):
	m = str(ML_monomers[i])
	#TODO: figure out if we want exact == here or "in" here (it changes the result) 
	index = [name for key, name in monomer_idx_dict.items() if m in key]
	if len(index) > 1: 
		print("too many?")
		too_many_ML_match_Pids.append(m)
		too_many_ML_match_Pidxs.append(index)
	if len(index) == 0:
		non_ML_match_Pids.append(m)
		non_ML_match_Pidxs.append(i)
	else: 
		index = index[0]
		ML_match_Pids.append(m)
		ML_match_idxs.append(i)
		gene_id = idx_gene_dict.get(index)
		ML_match_Gids.append(gene_id)
		gene_CN = idx_GCN_dict.get(index)
		ML_match_GCNids.append(gene_CN)
		
print(len(ML_match_Pids))
print(len(too_many_ML_match_Pidxs))
print(len(non_ML_match_Pidxs))

too many?
too many?
too many?
too many?
too many?
too many?
1093
6
0


In [91]:
too_many_ML_match_Pids

['BIOTIN-SYN-MONOMER',
 'GLUTKIN-MONOMER',
 'GLYCEROL-3-P-ACYLTRANSFER-MONOMER',
 'PKI-MONOMER',
 'PROTEIN-NRI',
 'THII-MONOMER']